# Labelling

We can use an algorithm that generates a mask for each image downloaded referenced in the csv file.

In [1]:
from generate_mask import process_db

data_path = "data/sentinel_2"

process_db(images_path=data_path,
           output_dir=data_path,
           upsample_factor=1,
           output_type="vector"
)

Processing tiles from data/sentinel_2
Processing tiles from data/sentinel_2


Alternatively, we can use scaneo to label the data by hand. In our case,  we use it just to check the masks that were generated and manually correct them if needed. 

In [ ]:
!scaneo --data data/sentinel_2

Environment file .env not found.
Running command: DATA=data/sentinel_2 uvicorn api:app --port 8000 --host localhost  --app-dir /home/coquarpj/miniforge3/envs/eotdl/lib/python3.9/site-packages/scaneo
Database already exists
INFO:     Started server process [623191]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:38254 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/assets/0.bf055a90.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/assets/3.2e6ca10d.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/assets/PaintPolygon.c10d0736.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/entry/start.d38ddb5e.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/chunks/index.a81f82fa.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:38254 - "GET /_app/immutable/chunks/singletons.853

We can finaly clean up the database by finaly removing all the images and corresponding metadata for which the labeling is unsuccessful. 

In [15]:
import json
from pathlib import Path

for mask_path in Path(data_path).glob("*geojson"):
    with open(mask_path) as src:
        mask_not_found = json.load(src)['features'] == []

        if mask_not_found:
            image_id = mask_path.name[:-15]
            for file_path in Path(data_path).glob(f"{image_id}*"):
                # Erase the files
                file_path.unlink()

